In [66]:
import pandas as pd
import json

# 1. Load the data
# Using sep='\t' because your file snippet indicates tab-separation
input_file = 'Ecoli_glcn_Solved_To_Acceptable_Level.fluxes.csv'
df = pd.read_csv(input_file, sep='\t')
#df1.columns=["Reaction_ID", 'Flux']

# 2. Inspect and Clean Columns
# Based on your snippet, the file might have an unnamed index column.
# We need to identify the column with IDs and the column with Fluxes.
# We assume the column with strings is the ID and the one with floats is the Flux.

# If pandas read the first row as a header and it looks like "0", we might need to adjust.
# Let's ensure we grab the right data. 
# Renaming columns assuming index 0 is ID and index 1 is Flux for safety:
if len(df.columns) >= 2:
    # Adjust column names for clarity
    df.columns.values[0] = 'Reaction_ID' 
    df.columns.values[1] = 'Flux'
else:
    print("Error: The file does not have enough columns.")

# 3. CRITICAL: Fix Reaction IDs for Escher
# Escher maps usually do NOT use the "R_" prefix (e.g., "EX_glc__D_e" instead of "R_EX_glc__D_e")
# We remove "R_" from the start of the string if it exists.
df['Reaction_ID'] = df['Reaction_ID'].astype(str).str.replace('^R_', '', regex=True)

In [67]:
df["Flux"][df["Reaction_ID"]=="EX_glcn_e"]

12   -0.155808
Name: Flux, dtype: float64

In [68]:
# 4. Convert to Escher Dictionary Format
# Escher expects a dictionary: { "Reaction_ID": Flux_Value }

alpha = (-7.2839/df["Flux"][df["Reaction_ID"]=="EX_glcn_e"]).item()
df["Flux"] = df["Flux"]*alpha
df["Flux"][df["Reaction_ID"]=="EX_glcn_e"]

12   -7.2839
Name: Flux, dtype: float64

In [69]:
escher_data = df.set_index('Reaction_ID')['Flux'].to_dict()

In [70]:
# 5. Save as JSON
output_file = 'escher_fluxes_pheflux.json'
with open(output_file, 'w') as f:
    json.dump(escher_data, f)

print(f"Success! Data saved to {output_file}")
print("--- Preview of data for Escher ---")
# Print first 5 items to verify IDs look correct (no 'R_' prefix)
print(dict(list(escher_data.items())[:5]))

Success! Data saved to escher_fluxes_pheflux.json
--- Preview of data for Escher ---
{'EX_cm_e': 6.293547879281514e-23, 'EX_cmp_e': -4.671132663858392e-07, 'EX_co2_e': 19.571894195618384, 'EX_cobalt2_e': -1.4159988319145322e-05, 'DM_4crsol_c': 0.0001311025168157572}


In [47]:
df["Flux"][df["Reaction_ID"]=="EX_glcn_e"]

12   -7.2839
Name: Flux, dtype: float64

In [37]:
%pwd

'/home/marcelo/jupyter/teraflux/fig5/Ecoli/results/pheflux'

In [62]:
-7.2839/df["Flux"][df["Reaction_ID"]=="EX_glcn_e"].item()

46.74930074121169